In [1]:
!pip install pandas numpy scikit-learn rdkit-pypi tqdm imbalanced-learn joblib matplotlib seaborn

!pip install xgboost lightgbm


ERROR: Could not find a version that satisfies the requirement rdkit-pypi (from versions: none)
ERROR: No matching distribution found for rdkit-pypi


In [4]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm


In [5]:
# Load dataset
df = pd.read_csv("chembl_ic50_single_protein_subset.csv")

# Show basic info
print(df.shape)
print(df.columns)
df.head()


(15272, 12)
Index(['Unnamed: 0', 'Molecule ChEMBL ID', 'Molecule Max Phase', 'Smiles',
       'pChEMBL Value', 'Potential Duplicate', 'Assay Description',
       'Target Type', 'Standard Relation', 'Standard Value', 'Standard Units',
       'Value'],
      dtype='object')


,Unnamed: 0,Molecule ChEMBL ID,Molecule Max Phase,Smiles,pChEMBL Value,Potential Duplicate,Assay Description,Target Type,Standard Relation,Standard Value,Standard Units,Value
0,0,CHEMBL48029,0,c1ccc(COc2ccc(CCNc3ccncc3)cc2)cc1,6.89,0.0,Displacement of [35S]MK-499 from hERG potassiu...,SINGLE PROTEIN,'=',130.0,nM,130.00
1,1,CHEMBL2158563,0,CCN(CC)CCOc1cc(-c2nc(SCC(=O)NC)nc3[nH]cc(C#N)c...,5.50,0.0,Displacement of [3H]-dofetilide from human ERG,SINGLE PROTEIN,'=',3140.0,nM,3.14
2,2,CHEMBL4102140,0,N#C[C@H]1CCOC[C@@H]1n1cc(C(N)=O)c(Nc2ccc(OC(F)...,4.60,0.0,Displacement of [35S]-MK499 from human ERG exp...,SINGLE PROTEIN,'=',25000.0,nM,25000.00
3,3,CHEMBL4096821,0,N#C[C@H]1CCOC[C@@H]1n1cc(C(N)=O)c(Nc2ccc(-n3cc...,4.88,0.0,Displacement of [35S]-MK499 from human ERG exp...,SINGLE PROTEIN,'=',13200.0,nM,13200.00
4,4,CHEMBL1771096,0,Cc1cnc(N2CCC(C3CCN(c4cc(C)nc(C#N)n4)CC3)CC2)cn1,4.97,0.0,Inhibition of human ERG,SINGLE PROTEIN,'=',10600.0,nM,10600.00


In [6]:
# Adjust column names as per your dataset
df = df[['canonical_smiles', 'standard_value']].dropna()

# Remove invalid or zero IC50 values
df = df[df['standard_value'] > 0]

# Create binary labels — lower IC50 means more active/toxic
df['label'] = np.where(df['standard_value'] <= 1000, 1, 0)

# Keep only SMILES + label
df = df[['canonical_smiles', 'label']]
df.rename(columns={'canonical_smiles': 'smiles'}, inplace=True)
df.head()


KeyError: "None of [Index(['canonical_smiles', 'standard_value'], dtype='object')] are in the [columns]"

In [ ]:
tqdm.pandas()

def smiles_to_morgan(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)

df["fingerprint"] = df["smiles"].progress_apply(smiles_to_morgan)
df = df.dropna(subset=["fingerprint"])


In [8]:
df = df[['Smiles', 'Standard Value']].dropna()

# Remove invalid or zero IC50 values
df = df[df['Standard Value'] > 0]

# Create binary labels — lower IC50 means more active/toxic
df['label'] = np.where(df['Standard Value'] <= 1000, 1, 0)

# Keep only SMILES + label
df = df[['Smiles', 'label']]

df.head()


,Smiles,label
0,c1ccc(COc2ccc(CCNc3ccncc3)cc2)cc1,1
1,CCN(CC)CCOc1cc(-c2nc(SCC(=O)NC)nc3[nH]cc(C#N)c...,0
2,N#C[C@H]1CCOC[C@@H]1n1cc(C(N)=O)c(Nc2ccc(OC(F)...,0
3,N#C[C@H]1CCOC[C@@H]1n1cc(C(N)=O)c(Nc2ccc(-n3cc...,0
4,Cc1cnc(N2CCC(C3CCN(c4cc(C)nc(C#N)n4)CC3)CC2)cn1,0


In [11]:
tqdm.pandas()

def smiles_to_morgan(Smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(Smiles)
    if mol is None:
        return None
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)

df["fingerprint"] = df["Smiles"].progress_apply(smiles_to_morgan)
df = df.dropna(subset=["fingerprint"])

[00:24:28] DEPRECATION WARNING: please use MorganGenerator                                   | 0/13405 [00:00<?, ?it/s]
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGenerator                           | 4/13405 [00:00<06:43, 33.25it/s]
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGenerator
[00:24:28] DEPRECATION WARNING: please use MorganGen

In [12]:
X = np.array([np.array(fp) for fp in df["fingerprint"]])
y = df["label"].values

print("Feature matrix shape:", X.shape)
print("Label distribution:\n", pd.Series(y).value_counts())


Feature matrix shape: (13405, 2048)
Label distribution:
 0    11127
1     2278
Name: count, dtype: int64


In [13]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
print("After SMOTE:", X_res.shape, pd.Series(y_res).value_counts())


After SMOTE: (22254, 2048) 1    11127
0    11127
Name: count, dtype: int64


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42, stratify=y_res
)


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Random Forest Accuracy: 0.9018198157717366
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      2226
           1       0.90      0.90      0.90      2225

    accuracy                           0.90      4451
   macro avg       0.90      0.90      0.90      4451
weighted avg       0.90      0.90      0.90      4451



In [16]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


XGBoost Accuracy: 0.9391148056616491
              precision    recall  f1-score   support

           0       0.95      0.93      0.94      2226
           1       0.93      0.95      0.94      2225

    accuracy                           0.94      4451
   macro avg       0.94      0.94      0.94      4451
weighted avg       0.94      0.94      0.94      4451



In [17]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)
lgb.fit(X_train, y_train)
y_pred_lgb = lgb.predict(X_test)

print("LightGBM Accuracy:", accuracy_score(y_test, y_pred_lgb))
print(classification_report(y_test, y_pred_lgb))


[LightGBM] [Info] Number of positive: 8902, number of negative: 8901
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.150441 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4044
[LightGBM] [Info] Number of data points in the train set: 17803, number of used features: 2022
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500028 -> initscore=0.000112
[LightGBM] [Info] Start training from score 0.000112
LightGBM Accuracy: 0.9420354976409796
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      2226
           1       0.94      0.94      0.94      2225

    accuracy                           0.94      4451
   macro avg       0.94      0.94      0.94      4451
weighted avg       0.94      0.94      0.94      4451



C:\Users\anshul\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
